In [9]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

## Funzione da ottimizzare

In [10]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
    V,CV = model[1:2]
    # se il modello è in 3d o 2d (guardo le righe di V, in 3d V è una 3xN, in 2d V è una 2xN)
    dim = size(V,1)
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
    #----------------------------------------------------------
    bboxes = [hcat(boundingbox(cell)...) for cell in cellpoints]
    xboxdict = coordintervals(1,bboxes)
    yboxdict = coordintervals(2,bboxes)
    # xs,ys are IntervalTree type
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = boxcovering(bboxes, 1, xs)
    ycovers = boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

    if dim == 3
        zboxdict = coordintervals(3,bboxes)
        zs = IntervalTrees.IntervalMap{Float64, Array}()
        for (key,boxset) in zboxdict
            zs[tuple(key...)] = boxset
        end
        zcovers = boxcovering(bboxes, 3, zs)
        covers = [intersect(pair...) for pair in zip(zcovers,covers)]
    end
    # remove each cell from its cover
    for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])
    end
    return covers
end

spaceindex (generic function with 1 method)

## Dipendenze della funzione

In [11]:
function boundingbox(vertices::Lar.Points)
   minimum = mapslices(x->min(x...), vertices, dims=2)
   maximum = mapslices(x->max(x...), vertices, dims=2)
   return minimum, maximum
end

function coordintervals(coord,bboxes)
    boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
    for (h,box) in enumerate(bboxes)
        key = box[coord,:]
        if haskey(boxdict,key) == false
            boxdict[key] = [h]
        else
            push!(boxdict[key], h)
        end
    end
    return boxdict
end

function boxcovering(bboxes, index, tree)
    covers = [[] for k=1:length(bboxes)]
    for (i,boundingbox) in enumerate(bboxes)
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        for x in iterator
            append!(covers[i],x.value)
        end
    end
    return covers
end

boundingbox (generic function with 1 method)

## 0) Benchmark vecchia funzione 

In [14]:
V = [0.0 2.0 2.0 -2.0 4.0 4.0 6.0 2.0 -1.0;
      0.0 2.0 0.0  2.0 4.0 6.0 6.0 6.0  3.0]
EV = [[1,2], [3,4], [5,6], [7,6], [8,9]]
@btime spaceindex((V,EV))

  109.097 μs (981 allocations: 53.19 KiB)


5-element Array{Array{Int64,1},1}:
 [2]
 [1]
 [4]
 [3]
 []

## 1) Controllo se la funzione è type unstable

In [15]:
@code_warntype spaceindex((V,EV))

Variables
  #self#::Core.Compiler.Const(spaceindex, false)
  model::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  @_3::Int64
  #20::var"#20#24"{Array{Float64,2},Array{Array{Int64,1},1}}
  #21::var"#21#25"
  #22::var"#22#26"
  #23::var"#23#27"
  V::Array{Float64,2}
  CV::Array{Array{Int64,1},1}
  dim::Int64
  cellpoints::Array{Array{Float64,2},1}
  bboxes::Array{_A,1} where _A
  xboxdict::OrderedDict{Array{Float64,1},Array{Int64,1}}
  yboxdict::OrderedDict{Array{Float64,1},Array{Int64,1}}
  xs::IntervalTrees.IntervalBTree{Float64,IntervalValue{Float64,Array},64}
  @_16::Union{Nothing, Tuple{Pair{Array{Float64,1},Array{Int64,1}},Int64}}
  ys::IntervalTrees.IntervalBTree{Float64,IntervalValue{Float64,Array},64}
  @_18::Union{Nothing, Tuple{Pair{Array{Float64,1},Array{Int64,1}},Int64}}
  xcovers::Array{Array{Any,1},1}
  ycovers::Array{Array{Any,1},1}
  covers::Array{Array{Any,1},1}
  zboxdict::OrderedDict{Array{Float64,1},Array{Int64,1}}
  zs::IntervalTrees.IntervalBTree{Float64,Interv

8 ──        (zboxdict = Main.coordintervals(3, bboxes))
│    %100 = IntervalTrees.IntervalMap::Core.Compiler.Const(IntervalTrees.IntervalBTree{K,IntervalValue{K,V},64} where V where K, false)
│    %101 = Core.apply_type(%100, Main.Float64, Main.Array)::Core.Compiler.Const(IntervalTrees.IntervalBTree{Float64,IntervalValue{Float64,Array},64}, false)
│           (zs = (%101)())
│    %103 = zboxdict::OrderedDict{Array{Float64,1},Array{Int64,1}}
│           (@_24 = Base.iterate(%103))
│    %105 = (@_24 === nothing)::Bool
│    %106 = Base.not_int(%105)::Bool
└───        goto #11 if not %106
9 ┄─ %108 = @_24::Tuple{Pair{Array{Float64,1},Array{Int64,1}},Int64}::Tuple{Pair{Array{Float64,1},Array{Int64,1}},Int64}
│    %109 = Core.getfield(%108, 1)::Pair{Array{Float64,1},Array{Int64,1}}
│    %110 = Base.indexed_iterate(%109, 1)::Core.Compiler.PartialStruct(Tuple{Array{Float64,1},Int64}, Any[Array{Float64,1}, Core.Compiler.Const(2, false)])
│           (key@_34 = Core.getfield(%110, 1))
│         

la funzione NON è type unstable in quanto ho nell'output la stringa:

    Body::Array{Array{Int64,1},1}
   